In [2]:
import numpy as np
import pandas as pd

import requests
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import base64


In [52]:
client_id = '7a2b21cebdb44bc282e6e415eaccf347'
client_secret = 'a276bff35be5492da4e8c8015ebcd5ae'

# Codifica las credenciales en base64
encoded = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

headers = {
    'Authorization': 'Basic ' + encoded
}

body = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=body)
access_token = auth_response.json()['access_token']

### Obtener albunes mas nuevos

In [65]:
headers = {
    'Authorization': 'Bearer ' + access_token
}

url = 'https://api.spotify.com/v1/browse/new-releases?market=ES'

response = requests.get(url, headers=headers)
data = response.json()


artist_list = []

if data.get('albums') and data['albums'].get('items'):
    for album in data['albums']['items']:
        for artist in album['artists']:
         
            artist_info = {
                'artist_name': artist['name'],
                'album_name': album['name'],
                'release_date': album['release_date'],
                'total_tracks': album['total_tracks'],
                'available_markets': album['available_markets'],
                'type': album['type'],     
            }

            artist_list.append(artist_info)

# Convertir la lista en DataFrame
df_artists = pd.DataFrame(artist_list)

# Mostrar el DataFrame
df_artists.head()




,artist_name,album_name,release_date,total_tracks,available_markets,type
0,GULEED,Cuando Menos Lo Espera,2023-07-07,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
1,Morad,Cuando Menos Lo Espera,2023-07-07,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
2,Ana Mena,bellodrama,2023-03-24,15,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
3,Myke Towers,LA VIDA ES UNA,2023-03-23,23,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album
4,Eladio Carrion,3MEN2 KBRN,2023-03-17,18,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",album


In [66]:

headers = {
    'Authorization': 'Bearer ' + access_token
}
top_tracks_url = "https://api.spotify.com/v1/playlists/37i9dQZEVXbNFJfN1Vw8d9/tracks"  
response = requests.get(top_tracks_url, headers=headers)

if response.status_code == 200:
    tracks_data = response.json()
    
    if 'items' not in tracks_data:
        print("La clave 'items' no se encuentra en la respuesta JSON:", tracks_data)
    else:
        tracks_list = []
        
        for item in tracks_data['items']:
            
            track_info = {
                'track_name': item['track']['name'],
                'artist_name': item['track']['artists'][0]['name'],
                'album_name': item['track']['album']['name'],
                'release_date': item['track']['album']['release_date'],
                'duration_ms': item['track']['duration_ms'],
                'popularity': item['track']['popularity'],
                
            }
            
            
            tracks_list.append(track_info)
        
        df_tracks = pd.DataFrame(tracks_list)
        
        print(df_tracks.head())
else:
    print("Error en la solicitud:", response.status_code, response.text)

{'added_at': '2024-03-19T16:34:16Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'}, 'href': 'https://api.spotify.com/v1/users/', 'id': '', 'type': 'user', 'uri': 'spotify:user:'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4ixNZvQuuzCDxBsxO9zE6L'}, 'href': 'https://api.spotify.com/v1/artists/4ixNZvQuuzCDxBsxO9zE6L', 'id': '4ixNZvQuuzCDxBsxO9zE6L', 'name': 'Gonzy', 'type': 'artist', 'uri': 'spotify:artist:4ixNZvQuuzCDxBsxO9zE6L'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/2O8vbr4RYPpk6MRA4fio7u'}, 'href': 'https://api.spotify.com/v1/artists/2O8vbr4RYPpk6MRA4fio7u', 'id': '2O8vbr4RYPpk6MRA4fio7u', 'name': 'SAIKO', 'type': 'artist', 'uri': 'spotify:artist:2O8vbr4RYPpk6MRA4fio7u'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/4SsVbpTthjScTS7U2hmr1X'}, 'href': 'https://api.spotify.com/v1/artists/4SsVbpTthjS